In [25]:
import plotly
print(plotly.__version__)

5.17.0


We will be using plotly 5.17.0 - it has already been included in the `requirements.txt`

If you wish to switch to the newest version (5.18), run  
`pip install --upgrade plotly` 

When running a plotly chart for the first time, you may find out that you are missing a library. In this case, in your virtual environment, run:  
`pip install --upgrade nbformat`

For exporting charts locally to static charts, an additional plotly dependency is needed, which is not installed automatically:  
`pip install --upgrade kaleido`  
It has been already included in the `requirements.txt` for the course

In [26]:
import seaborn as sns

data = sns.load_dataset('penguins')

data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


Let's create some aggregated data for the exercise: 

In [27]:
data_agg = data.groupby(['species','island','sex'])['body_mass_g'].count().rename('count').reset_index()
data_agg2 = data_agg.groupby('species')['count'].sum().reset_index()

In [28]:
data_agg2

,species,count
0,Adelie,146
1,Chinstrap,68
2,Gentoo,119


# Plotly charts as JSON

In [29]:
# import the IO functions 
import plotly.io as pio

# import low-level API
import plotly.graph_objects as go 

# import high-level API
import plotly.express as px 

### Method 1: Talking directly to the plotly.js API

In [30]:
# all about the chart type and data characteristics 

datatrace = {
    'type': 'bar', 
    'x': ['Adelie', 'Chinstrap', 'Gentoo'], 
    'y': [146, 68, 119]
}

In [31]:
# all about the chart layout: title, grid, colors, margins etc.

layout = {
    'title': 'Penguin counts'
}

In [32]:
figdict = {'data': datatrace, 
          'layout': layout}

In [33]:
figdict

{'data': {'type': 'bar',
  'x': ['Adelie', 'Chinstrap', 'Gentoo'],
  'y': [146, 68, 119]},
 'layout': {'title': 'Penguin counts'}}

Pass the chart specification to Plotly IO: 

In [34]:
pio.show(figdict)

### Method 2: Pass the JSON chart specification to Plotly graph_objects: 

In [35]:
datatrace

{'x': ['Adelie', 'Chinstrap', 'Gentoo'], 'y': [146, 68, 119], 'type': 'bar'}

In [36]:
layout

{'title': 'Penguin counts'}

In [37]:
f = go.Figure(data=datatrace, layout=layout)

f.show()

In [38]:
type(f)

plotly.graph_objs._figure.Figure

Advantage: we have now an object containing the whole chart specs: 

In [39]:
f.to_dict()

{'data': [{'x': ['Adelie', 'Chinstrap', 'Gentoo'],
   'y': [146, 68, 119],
   'type': 'bar'}],
 'layout': {'title': {'text': 'Penguin counts'},
  'template': {'data': {'histogram2dcontour': [{'type': 'histogram2dcontour',
      'colorbar': {'outlinewidth': 0, 'ticks': ''},
      'colorscale': [[0.0, '#0d0887'],
       [0.1111111111111111, '#46039f'],
       [0.2222222222222222, '#7201a8'],
       [0.3333333333333333, '#9c179e'],
       [0.4444444444444444, '#bd3786'],
       [0.5555555555555556, '#d8576b'],
       [0.6666666666666666, '#ed7953'],
       [0.7777777777777778, '#fb9f3a'],
       [0.8888888888888888, '#fdca26'],
       [1.0, '#f0f921']]}],
    'choropleth': [{'type': 'choropleth',
      'colorbar': {'outlinewidth': 0, 'ticks': ''}}],
    'histogram2d': [{'type': 'histogram2d',
      'colorbar': {'outlinewidth': 0, 'ticks': ''},
      'colorscale': [[0.0, '#0d0887'],
       [0.1111111111111111, '#46039f'],
       [0.2222222222222222, '#7201a8'],
       [0.3333333333333333, 

### Method 3: build up an go-object in an object-oriented way

In [40]:
fgo = go.Figure()

fgo.add_trace(
    go.Bar(
        x = ['Adelie', 'Chinstrap', 'Gentoo'],
        y = [146, 68, 119]
    )
)
# instead of declaring a chart "type", we use a predefined object included in the GO library, 
# which corresponds to a trace type

fgo.update_layout(
    title = 'Penguin counts'
)

Alternatively, we can also use a Layout object which is part of the GO library:

In [41]:
fgo2 = go.Figure(
    data = [
            go.Bar(
                x = ['Adelie', 'Chinstrap', 'Gentoo'],
                y = [146, 68, 119]
            )
            ], 
    layout = go.Layout(
                title = 'Penguin counts'
                )
)

fgo2.show()

### Method 4: use a high-level API (plotly express)

Instead of building charts from blocks and objects corresponding to some abstractions (chart type, layout), we can use a high-level API where one chart type with data corresponds to one function

In [42]:
fpx = px.bar(
        x = ['Adelie', 'Chinstrap', 'Gentoo'], 
        y = [146, 68, 119], 
        title = 'Penguin counts'
    )

In [43]:
fpx.show()

Notice: now we got the x,y axis labels. The two rendered charts are not entirely the same - plotly express and plotly GO have slightly different defaults 

The price we pay for this simplicity is that we cannot re-use the low-level objects defined above: 

In [44]:
# px.bar(**datatrace, **layout) # this won't work

Here we no longer have to (or can) specify "type" - we choose type by choosing `px.bar` already, similar to choosing a `go.Bar()` object 

Bad news: some of the arguments don't match between the APIs - but at least they are similar enough

Good news: Every px-generated chart is actually a graph_object: 

In [45]:
type(fpx)

plotly.graph_objs._figure.Figure

In [46]:
type(fgo)

plotly.graph_objs._figure.Figure

However, we see that fpx does much more "thinking" for us upfront and the json contains some values which we didn't encode ourselves: 

In [47]:
# plotly express
fpx.to_dict()['data']

[{'alignmentgroup': 'True',
  'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
  'legendgroup': '',
  'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
  'name': '',
  'offsetgroup': '',
  'orientation': 'v',
  'showlegend': False,
  'textposition': 'auto',
  'x': array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object),
  'xaxis': 'x',
  'y': array([146,  68, 119], dtype=int64),
  'yaxis': 'y',
  'type': 'bar'}]

In [48]:
# plotly go 
fgo.to_dict()['data']

[{'x': ['Adelie', 'Chinstrap', 'Gentoo'], 'y': [146, 68, 119], 'type': 'bar'}]

Luckily, we can combine the two such that we can create a base chart using px-API and modify it using go-API or JSON-API if we wish:

In [49]:
fpx

In [50]:
fpx.update_traces({'marker':{'color':'red'}})

# Changing chart types

Method 1: change the `'type'` property

In [51]:
datatrace = {
    'x': ['Adelie', 'Chinstrap', 'Gentoo'], 
    'y': [146, 68, 119], 
    'type': 'scatter' # try 'bar'
}

layout = {
    'title': 'Penguin counts'
}

figdict = {'data': datatrace, 
          'layout': layout}

go.Figure(**figdict)

<div style="background: red; color: white; font-size: 20px; text-align: center; padding: 1em; font-weight: bold;">You should never use a line chart like this! A line should never connect the categories!!!</div>

Also, notice that the line chart is created using a `scatter` function - in your project, describe which chart types you have decided to use using the concept name, not a function name - that is, do not call the line chart a scatter chart even if it was created using a "scatter" function

In [52]:
datatrace = {
    'x': ['Adelie', 'Chinstrap', 'Gentoo'], 
    'y': [146, 68, 119], 
    'type': 'scatter', # 'bar' will give the error now
    'mode': 'markers', 
    'marker': {'size': 20}
}

layout = {
    'title': 'Penguin counts'
}

figdict = {'data': datatrace, 
          'layout': layout}

go.Figure(**figdict)

Method 2: GO-approach - change go.Bar to another object

In [53]:
fgo = go.Figure()

fgo.add_trace(
    go.Scatter(
        x = ['Adelie', 'Chinstrap', 'Gentoo'],
        y = [146, 68, 119], 
        mode = 'markers', 
        marker_size = 20
    )
)
# again some properties are only valid for one chart type 

fgo.update_layout(
    title = 'Penguin counts'
)

# Combining Plotly with Pandas

Until now, we provided data by hand. How can we make Plotly work with Pandas? Luckily, it's very easy, even if we use a plotly.js approach

In [54]:
data_agg2

,species,count
0,Adelie,146
1,Chinstrap,68
2,Gentoo,119


Feed pandas series as `x`, `y`: 

In [55]:
datatrace = {
    'type': 'bar', 
    'x': data_agg2['species'], 
    'y': data_agg2['count']
}

layout = {
    'title': 'Penguin counts'
}

figdict = {'data': datatrace, 
          'layout': layout}

go.Figure(**figdict)

In [56]:
figdict

{'data': {'x': 0       Adelie
  1    Chinstrap
  2       Gentoo
  Name: species, dtype: object,
  'y': 0    146
  1     68
  2    119
  Name: count, dtype: int64,
  'type': 'bar'},
 'layout': {'title': 'Penguin counts'}}

It worked although we passed pd.Series objects as x, y

Plotly express API accepts dataframes similarly to seaborn 

In [57]:
fpx2 = px.scatter(data_frame=data_agg2, 
                  x='species', y='count', ) 
# notice an inconsistency: we no longer have to remove the lines! px.scatter is scatter-by-default
fpx2.show()

However the df support may be sometimes unintuitive: 

In [58]:
# however feeding a data_frame makes plotly expect size as one of the df columns... :( 
fpx2 = px.scatter(data_frame=data_agg2, 
                  x='species', y='count', 
                 size=20) 
fpx2.show()

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of ['species', 'count'] but received: 20

Workaround 1: input the columns directly and input the size as an array of sizes

In [59]:
fpx2 = px.scatter(
                  x=data_agg2['species'], y=data_agg2['count'], 
                  size=[20, 20, 20]) 

## alternatives: 
# size = [20 for x in range(data_agg2.shape[0])]
# size = data_agg2.apply(lambda x: 20, axis=1)

fpx2.show()

Workaround 2: create a new column in the dataframe. This will pollute the dataframe, but it is a consequence of the design choice by Plotly creators - they wanted to have an easy option to make bubble charts 

In [60]:
data_agg2['px_size'] = 20
fpx2 = px.scatter(data_frame=data_agg2,
                  x='species', y='count', 
                  size='px_size') 
fpx2.show()

# Overplotting multiple traces: 

In [61]:
datatrace1 = {
    'type': 'scatter', 
    'mode': 'markers',
    'marker': {'size': 20},
    'x': data_agg2['species'], 
    'y': data_agg2['count']
}

datatrace2 = {
    'type': 'bar', 
    'x': data_agg2['species'], 
    'y': data_agg2['count']
}


layout = {
    'title': 'Penguin counts'
}

figdict = {'data': [datatrace1, datatrace2], # create a list of traces that can be different chart types
          'layout': layout}

go.Figure(**figdict)

In a go-API:

In [62]:
f2 = go.Figure()

f2.add_trace(datatrace1)
f2.add_trace(datatrace2)
f2.update_layout(layout)

In [63]:
# watch out: this modifies the global object living in the memory: 
f2.add_trace(datatrace1)

This approach will be useful when embedding charts in Dash apps. 

In the px-API: 

In [64]:
fpx2 = px.scatter(data_frame=data_agg2,
                  x='species', y='count', 
                  size='px_size') 

fpx2.add_bar(x=data_agg2['species'], y=data_agg2['count'])

## compare with: 
# fpx2.add_bar(data_frame=data_agg2, x='species', y='count') 
## the data_frame=... approach doesn't work here :( 


In [65]:
# what happens if we change the sequence? 

fpx3 = px.bar(data_frame=data_agg2, x='species', y='count')
fpx3.add_scatter(x=data_agg2['species'], y=data_agg2['count'], mode='markers', marker_size=data_agg2['px_size']) 

# notice a change: size vs marker_size 

So the functions have different defaults. Also now **the legend doesn't include the first trace by default**. 

___

# Customizing and styling 

### Styling the markers and data traces

In [66]:
data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [67]:
# create a list that will contain all our data 
traces = []

for sp in data['species'].unique():
    
    # for each species, create a trace: 
    
    tmp_data = data.query('species == @sp')
    tmp_trace = {
        'x': tmp_data['flipper_length_mm'], 
        'y': tmp_data['body_mass_g'], 
        'type': 'scatter',
        'name': sp, 
        'mode': 'markers', 
        'marker': {'symbol': 'circle', 
                  'size': 10}
    }
    traces.append(tmp_trace)
    
figdict = {'data': traces} # I have 3 traces now
go.Figure(**figdict)

Good practice: declare styling outside of the chart generation

In [ ]:
## for Matplotlib we had: 

# species_styles = {
#     'Adelie': {'color': 'tomato', 's': 100, 'alpha': 0.5}, 
#     'Chinstrap': {'color': 'dodgerblue', 'marker': 'v'}, 
#     'Gentoo': {'color': 'darkgray', 'edgecolor':'black'}
# }

In [68]:
species_styles = {
    'Adelie': {
        'marker': {'color': 'tomato', 'size': 15, 'opacity': 0.5},
    },
    'Chinstrap': {
        'marker':
            {'color': 'dodgerblue', 'symbol': 'triangle-down'}, 
    },
    'Gentoo': {
        'marker': 
            {'color': 'darkgray', 
             'line': {
                 'color':'black', 
                 'width': 1}
            }
    }
}

In [70]:
traces = []
for sp in data['species'].unique():
    tmp_data = data.query('species == @sp')
    tmp_trace = {
        'x': tmp_data['flipper_length_mm'].tolist(), 
        'y': tmp_data['body_mass_g'].tolist(), 
        'type': 'scatter',
        'name': sp, 
        'mode': 'markers', 
    }
    tmp_trace.update(species_styles[sp]) # here we add additional keys to the dictionary
    traces.append(tmp_trace)
    
figdict = {'data': traces}
go.Figure(**figdict)

In go-API, this would be: 

In [71]:
fgo = go.Figure()

for sp in data['species'].unique():
    tmp_data = data.query('species == @sp')
    
    tmp_trace_go = go.Scatter(
        x = tmp_data['flipper_length_mm'].tolist(), 
        y = tmp_data['body_mass_g'].tolist(), 
        mode = 'markers', 
        name = sp, 
    )
    
    tmp_trace_go.update(species_styles[sp]) 
    # hurrah! we can mix dictionary and go.Scatter()! otherwise we would have to rewrite a lot 
    
    fgo.add_trace(tmp_trace_go)

fgo.show()

In [72]:
tmp_trace_go

Scatter({
    'marker': {'color': 'darkgray', 'line': {'color': 'black', 'width': 1}},
    'mode': 'markers',
    'name': 'Gentoo',
    'x': [211.0, 230.0, 210.0, 218.0, 215.0, 210.0, 211.0, 219.0, 209.0, 215.0,
          214.0, 216.0, 214.0, 213.0, 210.0, 217.0, 210.0, 221.0, 209.0, 222.0,
          218.0, 215.0, 213.0, 215.0, 215.0, 215.0, 216.0, 215.0, 210.0, 220.0,
          222.0, 209.0, 207.0, 230.0, 220.0, 220.0, 213.0, 219.0, 208.0, 208.0,
          208.0, 225.0, 210.0, 216.0, 222.0, 217.0, 210.0, 225.0, 213.0, 215.0,
          210.0, 220.0, 210.0, 225.0, 217.0, 220.0, 208.0, 220.0, 208.0, 224.0,
          208.0, 221.0, 214.0, 231.0, 219.0, 230.0, 214.0, 229.0, 220.0, 223.0,
          216.0, 221.0, 221.0, 217.0, 216.0, 230.0, 209.0, 220.0, 215.0, 223.0,
          212.0, 221.0, 212.0, 224.0, 212.0, 228.0, 218.0, 218.0, 212.0, 230.0,
          218.0, 228.0, 212.0, 224.0, 214.0, 226.0, 216.0, 222.0, 203.0, 225.0,
          219.0, 228.0, 215.0, 228.0, 216.0, 215.0, 210.0, 219.0, 20

### Hover text 

Updating hover text: we can provide a template to `hovertemplate`, and address plotly variables as `{x}`, `{y]` etc. Additionally we can attach `text` and `customdata` to each point on the chart 

In [73]:
traces = []
for sp in data['species'].unique():
    tmp_data = data.query('species == @sp')
    tmp_trace = {
        'x': tmp_data['flipper_length_mm'].tolist(), 
        'y': tmp_data['body_mass_g'].tolist(), 
        'type': 'scatter',
        'name': sp, 
        'mode': 'markers', 
        'text': tmp_data['island'].tolist(), # I add the island name to each data point, although it is not used on the chart
        'hovertemplate': '<span style="font-weight:bold;">Island: %{text}</span><br>Species: '+sp+', Flipper %{x}mm, Mass %{y}g', 
        'hoverlabel': {
            'bgcolor': 'black', 
            'bordercolor': species_styles[sp]['marker']['color'] # you should probably try-except this
        } 
    }
    tmp_trace.update(species_styles[sp])
    traces.append(tmp_trace)
    
figdict = {'data': traces}
go.Figure(**figdict)

### Layout

We keep the traces, let's work on the layout: 

- https://plotly.com/python/axes/ 
- https://plotly.com/python/reference/layout/xaxis/ 

In [ ]:
# we will reuse the traces dictionary created above
# print(traces)

Axis and gridlines: 

In [74]:
layout = {
    'title': 'The penguin chart',
    'xaxis': {
        'range': [150,240],
        'showgrid': False, 
        'title': 'Flipper Length [mm]'
        }, 
    'yaxis': {
        'showline': True, 
        'linecolor': 'orange', 
        'linewidth': 2, 
        'gridcolor': 'black', 
        'title': 'Body Mass [g]', 
        'mirror': True
        }
}

figdict = {'data': traces, 'layout': layout}
go.Figure(**figdict)

Ticks styling and positions. Plotly templates: https://plotly.com/python/templates/

In [75]:
layout = {
    'title': 'The penguin chart',
    'xaxis': {
        'ticks': 'outside', # direction 
        'tick0': 0, #tick0 doesn't set the range! just counting start of the ticks
        'dtick': 25, # distance between the ticks 
        'ticklen': 8, # length of the little line 
        'tickwidth': 4, # width of the little line 
        'tickcolor': '#000000', 
        'mirror':'ticks' # place ticks above and below 
        }, 
    'yaxis': {
        }, 
    'template': 'seaborn', # we also have templates!
}

figdict = {'data': traces, 'layout': layout}
go.Figure(**figdict)

Colors, margins, padding, title and label plots, figure size: 

In [76]:
# experiment with different margin_val and pad_val
margin_val = 40
pad_val = 30

layout = {
    'title': {
        'text':'The penguin chart',
        'font': {
            'color': 'red', 
            'family':'Courier New',
            },
        'x': 0.5,
        'xanchor': 'center'
        },
    'xaxis': {
        }, 
    'yaxis': {
        'title': 'y-axis title',
        },
    
    'legend': {
        'bgcolor': 'white',
    },
    
    'paper_bgcolor': 'wheat', 
    'plot_bgcolor': 'lightblue',
    'font': {'family':'Times New Roman',},
    'width': 600, 
    'height': 300, 
    'margin': {'l': margin_val, 'r': margin_val, 'b': margin_val, 't': margin_val, 'pad': pad_val}
}

figdict = {'data': traces, 'layout': layout}
go.Figure(**figdict)

Customizing the modebar: buttons, modebar colors, export option

In [77]:
layout = {
    'title': 'The penguin chart',
    'xaxis': {
        }, 
    'yaxis': {
        'title': 'y-axis title'
        }, 
    'margin': {
        'r': 200
    }
}

config = {
    'modeBarButtonsToRemove': ['toggleSpikelines', "select2d", "lasso2d", "zoomIn2d", "zoomOut2d", "autoScale2d"],
    'displaylogo': False,
    'displayModeBar': True,
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'exported_plot',
        'height': None,
        'width': None,
        'scale': 4 # Multiply title/legend/axis/canvas sizes by this factor when exporting a chart 
        }
    }

modebar_config = {
        'orientation': 'h',
        'bgcolor': 'wheat',
        'color': 'blue',
        'activecolor': 'black',
}
layout['modebar'] = modebar_config

figdict = {'data': traces, 'layout':layout}
fc = go.Figure(**figdict)
fc.show(config=config) # we provide the config as argument to show()

# Subplots

In [85]:
trace1 = {
    'x': [1, 2, 3],
    'y': [4, 5, 6],
    'type': 'scatter', 
    'mode': 'lines+markers',
    'xaxis': 'x2',
    'yaxis': 'y2',
    }

trace2 = {
    'x': [20, 30, 40],
    'y': [50, 60, 70],

    'type': 'scatter', 
    'mode': 'markers'
    }

subplots_data = [trace1, trace2]

Our data contains two traces: 

In [81]:
subplots_data

[{'x': [1, 2, 3],
  'y': [4, 5, 6],
  'type': 'scatter',
  'mode': 'lines+markers',
  'xaxis': 'x2',
  'yaxis': 'y2'},
 {'x': [20, 30, 40], 'y': [50, 60, 70], 'type': 'scatter', 'mode': 'markers'}]

Define grid with 2 slots for charts - plotly will arrange the list of traces into those slots 

In [86]:
subplots_layout = {
    'grid': {
        'rows': 1, 
        'columns': 2, 
        'pattern': 'independent'},
      }

figdict = {'data': subplots_data, 'layout':subplots_layout}
go.Figure(**figdict)

We can add charts as insets in arbitrary places: 

In [87]:
# alternative layout with custom placement as an inset: 
subplots_layout_inset = {
    'yaxis2': {
        'domain': [0.1, 0.4],
        'anchor': 'x2'
        },
    'xaxis2': {
        'domain': [0.1, 0.2],
        'anchor': 'y2'
        }
    }

figdict = {'data': subplots_data, 'layout':subplots_layout_inset}
go.Figure(**figdict)

## Annotations

Using characters instead of symbols: `mode: text`

In [88]:
traces_text = []
for sp in data['species'].unique():
    tmp_data = data.query('species == @sp')
    tmp_trace = {
        'x': tmp_data['flipper_length_mm'], 
        'y': tmp_data['body_mass_g'], 
        'type': 'scatter',
        'name': sp, 
        'text': sp[0],
        'mode': 'text', # here we tell plotly to draw whatever is stored as `text` property
        'textposition': 'bottom center',
        'textfont': {'size': 18, 'color': species_styles[sp]['marker']['color']}
    }
    traces_text.append(tmp_trace)
    
figdict = {'data': traces_text}
ftext = go.Figure(**figdict)
ftext.show()

https://plotly.com/python/reference/layout/annotations/ 

https://plotly.com/python/shapes/

Adding arrows with annotations 

In [89]:
anno1 = {
    'ax': 200, # start of the arrow / text position  
    'ay': 5500, 
    'axref': 'x', 
    'ayref': 'y',
    
    'x': 185, # end of the arrow  
    'y': 3500, 
    'xref': 'x', 
    'yref': 'y',     
    
    'text': 'Penguins!',
    'font': {'size': 16, 'color': 'black'}, 
    'arrowhead': 2, 
    'arrowsize': 1, 
    'arrowwidth': 2, 
    'arrowcolor': '#128391',

}
ftext.add_annotation(anno1)

Drawing custom shapes: 

In [90]:
shape_coords =[ (180, 4000), (200, 5000), (205, 3500), (190,2800), (180,3000)] 
# if we want a closed shape, we should repeat the first point at the end

shape_x = [x[0] for x in shape_coords]
shape_y = [x[1] for x in shape_coords]

shape1 = {
    'x': shape_x, 
    'y': shape_y, 
    'fill': 'toself'
}

ftext.add_scatter(**shape1)
# ftext.add_scatter(x=shape_x, y=shape_y, fill="toself")

Notice that we are adding incrementally and our chart object remembers all the `add_...` calls that we made 

# Heatmaps and histograms

1D and 2D Histograms can be constructed using a pre-built histogram functions (https://plotly.com/python/histograms/ and https://plotly.com/python/2D-Histogram/) 

However here we look at heatmaps, because they are more basic  

Read the data, create the 2D histogram using numpy 

In [91]:
import pandas as pd
import numpy as np 
data2 = pd.read_csv('ALB_2020-03.csv', index_col=[0])
data2['DATE_TIME'] = pd.to_datetime(data2['DATE_TIME'])
nbins = 30
H, xedges, yedges = np.histogram2d(x=data2['DOWNSTREAM'], y=data2['UPSTREAM'], bins=nbins)

Create the histogram from the raw data using plotly `px.density_heatmap()`: 

In [92]:
fig_hist1 = px.density_heatmap(data_frame=data2, x="DOWNSTREAM", y="UPSTREAM", 
                  nbinsx=nbins, nbinsy=nbins, histfunc="count")
fig_hist1.update_layout(width=500, height=500) # make tiles square-ish
# fig_hist1.update_layout(yaxis = dict(scaleanchor = 'x')) # make pixels square 

Compare with the histogram calculated by numpy: 

In [93]:
fh = px.imshow(H.T, x=xedges[:-1], y=yedges[:-1], aspect='auto')
fh.update_layout(width=500, height=500)
fh.update_yaxes(autorange=None) # so that the y-axis is not flipped 

You may notice that the histogram data is actually different! It happened because plotly wanted to be "smart" about the bin widths. This is why I prefer to do the calculations myself 

# Colors 

Plotly has a collection of built-in color scales: https://plotly.com/python/builtin-colorscales/ 

We already know many of them, because they base on Cbrewer colormaps

In [94]:
fh = px.imshow(H.T, x=xedges[:-1], y=yedges[:-1], aspect='auto', color_continuous_scale='gray')
fh.update_layout(width=500, height=500)
fh.update_yaxes(autorange=None)

Colormaps live under `px.colors.[sequential,qualitative]`

In [95]:
px.colors.sequential.gray_r

['rgb(254, 254, 253)',
 'rgb(224, 224, 223)',
 'rgb(197, 197, 195)',
 'rgb(171, 171, 170)',
 'rgb(146, 146, 145)',
 'rgb(124, 123, 122)',
 'rgb(102, 101, 101)',
 'rgb(81, 80, 80)',
 'rgb(59, 59, 59)',
 'rgb(38, 38, 38)',
 'rgb(16, 16, 16)',
 'rgb(0, 0, 0)']

In [96]:
px.colors.sequential.Plasma

['#0d0887',
 '#46039f',
 '#7201a8',
 '#9c179e',
 '#bd3786',
 '#d8576b',
 '#ed7953',
 '#fb9f3a',
 '#fdca26',
 '#f0f921']

In [97]:
px.colors.qualitative.Plotly

['#636EFA',
 '#EF553B',
 '#00CC96',
 '#AB63FA',
 '#FFA15A',
 '#19D3F3',
 '#FF6692',
 '#B6E880',
 '#FF97FF',
 '#FECB52']

### Custom color scale: 

Equal distances: provide a list of colors 

In [98]:
fh = px.imshow(H.T, x=xedges[:-1], y=yedges[:-1], aspect='auto', 
               color_continuous_scale=['lightgray', 'gray', 'black'])
fh.update_layout(width=500, height=500)
fh.update_yaxes(autorange=None)

In [99]:
fh = px.imshow(H.T, x=xedges[:-1], y=yedges[:-1], aspect='auto', 
               color_continuous_scale=['rgb(12,14,20)', 'rgb(125,216,250)'])
fh.update_layout(width=500, height=500)
fh.update_yaxes(autorange=None)

Provide a list of tuples if the colors are to be changed in a particular proportion, e.g. a particular pure color should appear at 70\% of the max value:

In [100]:
fh = px.imshow(H.T, x=xedges[:-1], y=yedges[:-1], aspect='auto', 
               color_continuous_scale=[(0,'wheat'), (0.7, 'green'), (1, 'black')])

fh.update_layout(width=500, height=500)
fh.update_yaxes(autorange=None)

To make a discrete color scale or a discrete step, create 2 tuples where two colors correspond to the same \%. Can be used for masking/highlighting a particular range of values etc. 

In [101]:
fh = px.imshow(H.T, x=xedges[:-1], y=yedges[:-1], aspect='auto', 
               color_continuous_scale=[(0,'wheat'), (0.2, 'wheat'), (0.2, 'red'), (1, 'black')])

fh.update_layout(width=500, height=500)
fh.update_yaxes(autorange=None)

# Exporting the charts

Let's go back to one of the charts 

In [102]:
ftext

### Interactive charts

In [ ]:
# include the JS library 
ftext.write_html('exported_chart.html')

In [ ]:
# refer to the JS library, include only the chart spec 
ftext.write_html('exported_chart_online.html', include_plotlyjs='cdn')

Chart can be also exported to JSON file and rendered elsewhere: 

In [ ]:
chart_json = ftext.to_json()

In [ ]:
ftext.write_json('exported_chart.json')

### Static charts

Charts can be saved from the interactive view by clicking on a modebar button

To do the static export from the command line, we need another library: 
`pip install kaleido`

In [ ]:
ftext.write_image('exported_chart.png')

Exports to png, pdf etc. are possible

We can overwrite the image dimensions specified upon creating the chart, and instead of `dpi` here we set the `scale` to achieve higher resolution

In [ ]:
ftext.write_image('exported_chart2.png', width=600, height=350, scale=4)